<a href="https://colab.research.google.com/github/imdeepmind/CrowdCounter/blob/master/Crowd_Counter_(Regression).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Getting the dataset into proper place
!mkdir data
!cp '/content/drive/My Drive/datasets/dataset.zip' ./
!unzip -qq dataset.zip -d ./data/
!rm dataset.zip

In [32]:
from sklearn.model_selection import train_test_split
import pandas as pd


In [33]:
# Reading data
data = pd.read_csv("./data/processed.csv")

data.head()

,image,count
0,seq_000689.jpg,35
1,seq_000516.jpg,41
2,seq_000217.jpg,41
3,seq_000870.jpg,44
4,seq_000324.jpg,41


In [35]:
train, test = train_test_split(data, test_size=0.1, random_state=1969)

train.head()

,image,count
261,seq_000604.jpg,18
962,seq_000537.jpg,31
592,seq_000526.jpg,25
884,seq_000801.jpg,25
84,seq_000954.jpg,31


In [36]:
train_generator = ImageDataGenerator(rotation_range=180, zoom_range=1.0, horizontal_flip=True, vertical_flip=True, rescale=1/255., validation_split=0.1)
test_generator = ImageDataGenerator(rescale=1/255.)


In [37]:
train_data = train_generator.flow_from_dataframe(train, 
                                                 directory='./data/frames/frames/', 
                                                 x_col='image', 
                                                 y_col='count', 
                                                 target_size=(256, 256),
                                                 color_mode='rgb',
                                                 class_mode='raw',
                                                 batch_size=32,
                                                 shuffle=True)

test_data = test_generator.flow_from_dataframe(test, 
                                                 directory='./data/frames/frames/', 
                                                 x_col='image', 
                                                 y_col='count', 
                                                 target_size=(256, 256),
                                                 color_mode='rgb',
                                                 class_mode='raw',
                                                 batch_size=32,
                                                 shuffle=True)


Found 1800 validated image filenames.
Found 200 validated image filenames.
